## Pydantic Basics: Creating and Using Models

Pydantic models are Foundations of Data Validation in Python.
- Uses python Type Annotation to define the structure and validate data at runtime.


In [16]:
from pydantic import BaseModel

In [17]:
from dataclasses import dataclass

@dataclass
class Person():
    name :str
    age:int
    city:str
    country:str


person1 = Person(name="Kirtan",age=21,city="Debrecen",country="Hungary")

print(person1)
type(person1)


Person(name='Kirtan', age=21, city='Debrecen', country='Hungary')


__main__.Person

In [19]:
person2 = Person(name="Ramwani",age="25",city="debrecen",country=+36)   ## dataclass models no  validation error
print(person2)

Person(name='Ramwani', age='25', city='debrecen', country=36)


In [20]:
class Person(BaseModel):
    name :str
    age:int
    city:str
    country:str


person1 = Person(name="Kirtan",age=21,city="Debrecen",country="Hungary")

print(person1)
type(person1)

name='Kirtan' age=21 city='Debrecen' country='Hungary'


__main__.Person

In [21]:
person2 = Person(name="Ramwani",age="25",city="debrecen",country=+36)   ## base models validation error


ValidationError: 1 validation error for Person
country
  Input should be a valid string [type=string_type, input_value=36, input_type=int]
    For further information visit https://errors.pydantic.dev/2.11/v/string_type

## 2- Models with Optional fields

add optional fields using Pythons's optional type

In [24]:
from typing import Optional
class Employee(BaseModel):
    id:int
    name:str
    department:str
    salary:Optional[float] = None   # Optional with default value None
    is_active:Optional[bool] = True # Optional with default value True


Employee1= Employee

In [26]:
# Examples without optional value
employee1= Employee(id=1,name="Kirtan",department="IT")
print(employee1)

id=1 name='Kirtan' department='IT' salary=None is_active=True


In [27]:
# Examples with optional value
employee2= Employee(id=2,name="Ramwani",department="Finance",salary=60000.00,is_active=False)
print(employee2)

id=2 name='Ramwani' department='Finance' salary=60000.0 is_active=False


In [28]:
employee3= Employee(id=2,name="Ramwani",department="Finance",salary=60000,is_active=False) ## automatic type casting
print(employee3)

id=2 name='Ramwani' department='Finance' salary=60000.0 is_active=False


In [29]:
from pydantic import BaseModel
from typing import List


class Classroom(BaseModel):
    room_number:str
    students:List[str]  ## list of strings i-e student names
    capacity:int


In [31]:
# create classroom
classroom1 = Classroom(
    room_number = "A101",
    students=['Alice','Bob','Charlie'],
    capacity=30

)

print(classroom1)

room_number='A101' students=['Alice', 'Bob', 'Charlie'] capacity=30


In [32]:
classroom2 = Classroom(
    room_number = "A101",
    students=('Alice','Bob','Charlie'),  # typecasting of list to tuple
    capacity=30

)

print(classroom2)

room_number='A101' students=['Alice', 'Bob', 'Charlie'] capacity=30


In [36]:
try:   
    classroom3 = Classroom(room_number="A102",students=(1,2,3),capacity=30)
    print(classroom3)
except ValueError as e:
    print("Error:" , e)



Error: 3 validation errors for Classroom
students.0
  Input should be a valid string [type=string_type, input_value=1, input_type=int]
    For further information visit https://errors.pydantic.dev/2.11/v/string_type
students.1
  Input should be a valid string [type=string_type, input_value=2, input_type=int]
    For further information visit https://errors.pydantic.dev/2.11/v/string_type
students.2
  Input should be a valid string [type=string_type, input_value=3, input_type=int]
    For further information visit https://errors.pydantic.dev/2.11/v/string_type


### 4. Model with nested models

Creating complex struucture with nested models


In [42]:
from pydantic import BaseModel

class Address(BaseModel):
    street:str
    city:str
    zip_code:int

class Customer(BaseModel):
    customer_id : int
    name : str
    email :str
    address : Address # nested model


In [46]:
## create a customer with nested address 

customer1 = Customer(
    customer_id=1,
    name="Kirtan",
    email="ramwanikirtan@gmail.com",
    address={
        "street" :"123 Main st",
        "city" :'Debrecen',
        "zip_code":"4029"  ## type casted
    }

)

print(customer1)

customer_id=1 name='Kirtan' email='ramwanikirtan@gmail.com' address=Address(street='123 Main st', city='Debrecen', zip_code=4029)


## Pydantic fields : Customization and constraints

The field function in pydantic enhances model fields beyond basic type hints by allowing you to specify validation rules,default values,aliases and more.


In [47]:
from pydantic import BaseModel,Field

class Item(BaseModel):
    name:str= Field(min_length=3,max_length=50)
    price:float= Field(gt=0,le=1000) ## greater than 0 and less than or equal to 1000
    quantity:int= Field(ge=0)  ## greater than or equal to 0


In [48]:
 ## valid instance

item1 = Item(name='Laptop',price=650,quantity=10)
print(item1)

name='Laptop' price=650.0 quantity=10


In [49]:
  ## Invalid instance

item2 = Item(name='Laptop',price=1200,quantity=10)
print(item2)

ValidationError: 1 validation error for Item
price
  Input should be less than or equal to 1000 [type=less_than_equal, input_value=1200, input_type=int]
    For further information visit https://errors.pydantic.dev/2.11/v/less_than_equal

In [51]:
## Fields with Default value


class User(BaseModel):
    username: str = Field(...,description="Unique username")
    email: str = Field(default_factory=lambda:"user@example.com",description="User email address")
    age: int = Field(default=18,ge=0,description="User age,defaults to 18")




In [54]:
## Examples
user1 = User(username="Kirtan")
print(user1) 

## User with all fields
user2 = User(username="Kirtan",email="ramwanikirtan@gmail.com",age=21)
print(user2) 

username='Kirtan' email='user@example.com' age=18
username='Kirtan' email='ramwanikirtan@gmail.com' age=21


In [55]:
print(User.model_json_schema())

{'properties': {'username': {'description': 'Unique username', 'title': 'Username', 'type': 'string'}, 'email': {'description': 'User email address', 'title': 'Email', 'type': 'string'}, 'age': {'default': 18, 'description': 'User age,defaults to 18', 'minimum': 0, 'title': 'Age', 'type': 'integer'}}, 'required': ['username'], 'title': 'User', 'type': 'object'}
